# Stage 1 Architecture Comparison: LNN vs CNN vs LSTM vs Transformer
## LOS/NLOS Classification from Raw CIR

Compares **4 model architectures** on the combined 3600-sample dataset under **identical conditions**:

| Model | Readout (canonical) | Inductive Bias |
|---|---|---|
| **LNN** (DualCircuit_PI_HLNN) | Attention-pooled ODE trajectory | Continuous-time dynamics, physics priors |
| **CNN** (1D-CNN) | Global Average Pooling | Local patterns, translation invariance |
| **LSTM** | Last hidden state | Sequential gating, selective memory |
| **Transformer Encoder** | [CLS] token | All-pairs self-attention, positional encoding |

**Controlled variables** (identical across all models):
- Same inputs: 60-sample CIR window + FP_AMPL1/2/3 conditioning
- Same classifier head: Linear(64\u219232) \u2192 SiLU \u2192 Dropout \u2192 Linear(32\u21921) \u2192 Sigmoid
- Same training: BCE loss, AdamW, cosine LR with warmup, grad clipping
- Same data splits, same seeds, same evaluation metrics

Each architecture uses its own **canonical readout** \u2014 the readout is integral to the architecture's inductive bias.

**Evaluation**: Single 70/15/15 split + Stratified 5-Fold CV

In [ ]:
CONFIG = {
    "pre_crop": 10, "post_crop": 50, "total_len": 60,
    "search_start": 740, "search_end": 890,
    "hidden_size": 32, "input_size": 1, "dropout": 0.2, "ode_unfolds": 6,
    "batch_size": 64, "max_epochs": 50, "lr": 1e-3,
    "weight_decay": 1e-4, "warmup_epochs": 3, "patience": 40,
    "grad_clip": 1.0, "val_ratio": 0.15, "test_ratio": 0.15, "seed": 42,
}
DATA_DIR = "../dataset/channels/"

# ── Model registry ──────────────────────────────────────────────────────
MODEL_CONFIGS = {
    'LNN': {
        'class': 'DualCircuit_PI_HLNN',
        'kwargs': {
            'input_size': CONFIG['input_size'],
            'hidden_size': CONFIG['hidden_size'],
            'dropout': CONFIG['dropout'],
            'ode_unfolds': CONFIG['ode_unfolds'],
        },
    },
    'CNN': {
        'class': 'CNN1D_Classifier',
        'kwargs': {'dropout': CONFIG['dropout']},
    },
    'LSTM': {
        'class': 'LSTM_Classifier',
        'kwargs': {'lstm_hidden': 48, 'dropout': CONFIG['dropout']},
    },
    'Transformer': {
        'class': 'TransformerEncoder_Classifier',
        'kwargs': {
            'd_model': 32, 'nhead': 4, 'num_layers': 2,
            'dim_feedforward': 64, 'dropout': CONFIG['dropout'],
        },
    },
}

MODEL_COLORS = {
    'LNN': '#e74c3c',         # Red
    'CNN': '#3498db',         # Blue
    'LSTM': '#2ecc71',        # Green
    'Transformer': '#9b59b6', # Purple
}
MODEL_ORDER = ['LNN', 'CNN', 'LSTM', 'Transformer']

In [ ]:
import copy, math, contextlib, io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    confusion_matrix, ConfusionMatrixDisplay,
    classification_report, roc_curve, auc
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")
torch.manual_seed(CONFIG["seed"])
np.random.seed(CONFIG["seed"])

---
## Section 1: Data Loading & Preprocessing

In [ ]:
def get_roi_alignment(sig,
                      search_start=CONFIG['search_start'],
                      search_end=CONFIG['search_end']):
    region = sig[search_start:search_end]
    if len(region) == 0:
        return np.argmax(sig)
    peak_idx = search_start + np.argmax(region)
    peak_val = sig[peak_idx]
    noise_section = sig[:search_start]
    if len(noise_section) > 10:
        threshold = max(np.mean(noise_section) + 3*np.std(noise_section), 0.05*peak_val)
    else:
        threshold = 0.05 * peak_val
    leading_edge = peak_idx
    for i in range(peak_idx, max(search_start - 20, 0), -1):
        if sig[i] < threshold:
            leading_edge = i + 1
            break
    return leading_edge


def _process_rows(df):
    """Shared CIR + FP_AMPL preprocessing."""
    PRE = CONFIG['pre_crop']; TOTAL = CONFIG['total_len']
    cir_cols = sorted([c for c in df.columns if c.startswith('CIR')],
                      key=lambda x: int(x.replace('CIR', '')))
    seqs, labels, fp_features = [], [], []
    for _, row in df.iterrows():
        sig = pd.to_numeric(row[cir_cols], errors='coerce').fillna(0).astype(float).values
        rxpacc = float(row.get('RXPACC', 128.0))
        if rxpacc > 0:
            sig = sig / rxpacc
        f1 = float(row.get('FP_AMPL1', 0)) / max(rxpacc, 1) / 64.0
        f2 = float(row.get('FP_AMPL2', 0)) / max(rxpacc, 1) / 64.0
        f3 = float(row.get('FP_AMPL3', 0)) / max(rxpacc, 1) / 64.0
        fp_features.append([f1, f2, f3])
        le = get_roi_alignment(sig)
        start = max(0, le - PRE); end = start + TOTAL
        if end > len(sig): end = len(sig); start = max(0, end - TOTAL)
        crop = sig[start:end]
        if len(crop) < TOTAL:
            crop = np.pad(crop, (0, TOTAL - len(crop)), mode='constant')
        lo, hi = crop.min(), crop.max()
        crop = (crop - lo) / (hi - lo) if hi > lo else np.zeros(TOTAL)
        seqs.append(crop); labels.append(float(row['Label']))
    X = np.array(seqs).reshape(-1, TOTAL, 1).astype(np.float32)
    y = np.array(labels).astype(np.float32)
    F = np.array(fp_features).astype(np.float32)
    return X, y, F


def load_cir_dataset(filepath):
    print(f'Loading: {filepath}')
    df = pd.read_csv(filepath)
    print(f'  Samples: {len(df)}')
    X, y, F = _process_rows(df)
    print(f'  Output: X={X.shape}, y={y.shape}, F={F.shape} | LOS={int((y==0).sum())}, NLOS={int((y==1).sum())}')
    return X, y, F

print("Data loading functions ready.")

---
## Section 2: Model Architectures

Four architectures with their canonical readouts:
- **LNN**: Dual ODE circuits \u2192 attention pooling \u2192 64-dim
- **CNN**: Conv1d stack \u2192 Global Average Pooling \u2192 64-dim
- **LSTM**: Recurrent processing \u2192 last hidden state \u2192 64-dim
- **Transformer**: Self-attention \u2192 [CLS] token output \u2192 64-dim

In [ ]:
# ==========================================
# LNN: DualCircuit_PI_HLNN (ODE-based)
# ==========================================
class PILiquidCell(nn.Module):
    def __init__(self, input_size, hidden_size, ode_unfolds=6):
        super().__init__()
        self.hidden_size = hidden_size
        self.ode_unfolds = ode_unfolds
        self.gleak = nn.Parameter(torch.empty(hidden_size).uniform_(0.001, 1.0))
        self.vleak = nn.Parameter(torch.empty(hidden_size).uniform_(-0.2, 0.2))
        self.cm    = nn.Parameter(torch.empty(hidden_size).uniform_(0.4, 0.6))
        self.w     = nn.Parameter(torch.empty(hidden_size, hidden_size).uniform_(0.001, 1.0))
        self.erev  = nn.Parameter(torch.empty(hidden_size, hidden_size).uniform_(-0.2, 0.2))
        self.mu    = nn.Parameter(torch.empty(hidden_size, hidden_size).uniform_(0.3, 0.8))
        self.sigma = nn.Parameter(torch.empty(hidden_size, hidden_size).uniform_(3, 8))
        self.sensory_w     = nn.Parameter(torch.empty(input_size, hidden_size).uniform_(0.001, 1.0))
        self.sensory_mu    = nn.Parameter(torch.empty(input_size, hidden_size).uniform_(0.3, 0.8))
        self.sensory_sigma = nn.Parameter(torch.empty(input_size, hidden_size).uniform_(3, 8))

    def forward(self, x_t, h_prev, dt=1.0):
        gleak     = F.softplus(self.gleak)
        cm        = F.softplus(self.cm)
        w         = F.softplus(self.w)
        sensory_w = F.softplus(self.sensory_w)
        sensory_gate    = torch.sigmoid(self.sensory_sigma * (x_t.unsqueeze(-1) - self.sensory_mu))
        sensory_current = (sensory_w * sensory_gate * x_t.unsqueeze(-1)).sum(dim=1)
        cm_t = cm / (dt / self.ode_unfolds)
        v = h_prev
        for _ in range(self.ode_unfolds):
            rg = torch.sigmoid(self.sigma.unsqueeze(0) * (v.unsqueeze(2) - self.mu.unsqueeze(0)))
            wg = w.unsqueeze(0) * rg
            w_num = (wg * self.erev.unsqueeze(0)).sum(dim=1)
            w_den = wg.sum(dim=1)
            v = (cm_t*v + gleak*self.vleak + w_num + sensory_current) / (cm_t + gleak + w_den + 1e-8)
            v = torch.clamp(v, -1.0, 1.0)
        return v, cm / (gleak + w_den + 1e-8)


class DualCircuit_PI_HLNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=32, dropout=0.4, ode_unfolds=6):
        super().__init__()
        self.hidden_size = hidden_size
        self.cell_los  = PILiquidCell(input_size, hidden_size, ode_unfolds)
        self.cell_nlos = PILiquidCell(input_size, hidden_size, ode_unfolds)
        self.fp_to_los_init  = nn.Linear(3, hidden_size)
        self.fp_to_nlos_init = nn.Linear(3, hidden_size)
        self.P_nlos2los = nn.Linear(hidden_size, hidden_size, bias=False)
        self.P_los2nlos = nn.Linear(hidden_size, hidden_size, bias=False)
        self.gate_los   = nn.Linear(hidden_size * 2, hidden_size)
        self.gate_nlos  = nn.Linear(hidden_size * 2, hidden_size)
        self.los_attn   = nn.Linear(hidden_size, 1)
        self.nlos_attn  = nn.Linear(hidden_size, 1)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size*2, hidden_size), nn.SiLU(),
            nn.Dropout(dropout), nn.Linear(hidden_size, 1), nn.Sigmoid()
        )

    def _run_circuits(self, x_seq, fp_features=None):
        batch_size, seq_len, _ = x_seq.size()
        if fp_features is not None:
            h_los  = 0.1 * torch.tanh(self.fp_to_los_init(fp_features))
            h_nlos = 0.1 * torch.tanh(self.fp_to_nlos_init(fp_features))
        else:
            h_los  = torch.zeros(batch_size, self.hidden_size, device=x_seq.device)
            h_nlos = torch.zeros(batch_size, self.hidden_size, device=x_seq.device)
        los_states, nlos_states = [], []
        tau_los_sum  = torch.zeros_like(h_los)
        tau_nlos_sum = torch.zeros_like(h_nlos)
        tau_los_hist_list, tau_nlos_hist_list = [], []
        for t in range(seq_len):
            x_t = x_seq[:, t, :]
            p_n2l = self.P_nlos2los(h_nlos); p_l2n = self.P_los2nlos(h_los)
            gl = torch.sigmoid(self.gate_los( torch.cat([h_los,  p_n2l], dim=1)))
            gn = torch.sigmoid(self.gate_nlos(torch.cat([h_nlos, p_l2n], dim=1)))
            h_los,  tau_los  = self.cell_los( x_t, h_los  + gl * p_n2l)
            h_nlos, tau_nlos = self.cell_nlos(x_t, h_nlos + gn * p_l2n)
            los_states.append(h_los.unsqueeze(1))
            nlos_states.append(h_nlos.unsqueeze(1))
            tau_los_sum  += tau_los
            tau_nlos_sum += tau_nlos
            tau_los_hist_list.append(tau_los.unsqueeze(1))
            tau_nlos_hist_list.append(tau_nlos.unsqueeze(1))
        los_all  = torch.cat(los_states, dim=1)
        nlos_all = torch.cat(nlos_states, dim=1)
        tau_los_mean  = tau_los_sum  / seq_len
        tau_nlos_mean = tau_nlos_sum / seq_len
        tau_los_hist  = torch.cat(tau_los_hist_list, dim=1)
        tau_nlos_hist = torch.cat(tau_nlos_hist_list, dim=1)
        return los_all, nlos_all, tau_los_hist, tau_nlos_hist, tau_los_mean, tau_nlos_mean

    def _pool_and_fuse(self, la, na):
        lw = F.softmax(self.los_attn(la).squeeze(-1),  dim=1).unsqueeze(-1)
        nw = F.softmax(self.nlos_attn(na).squeeze(-1), dim=1).unsqueeze(-1)
        return torch.cat([(la*lw).sum(1), (na*nw).sum(1)], dim=1)

    def forward(self, x_seq, fp_features=None, return_dynamics=False):
        los_all, nlos_all, tau_los_hist, tau_nlos_hist, tau_los_mean, tau_nlos_mean = \
            self._run_circuits(x_seq, fp_features=fp_features)
        pred = self.classifier(self._pool_and_fuse(los_all, nlos_all))
        if return_dynamics:
            return pred, los_all, nlos_all, tau_los_hist, tau_nlos_hist, tau_los_mean, tau_nlos_mean
        return pred, tau_los_mean, tau_nlos_mean

    def embed(self, x_seq, fp_features=None):
        los_all, nlos_all, _, _, _, _ = self._run_circuits(x_seq, fp_features=fp_features)
        return self._pool_and_fuse(los_all, nlos_all)

print('LNN: DualCircuit_PI_HLNN defined.')

In [ ]:
# ==========================================
# BASE CLASSIFIER (shared FP conditioning + classifier head)
# ==========================================
class BaseClassifier(nn.Module):
    """Base class for non-LNN classifiers.

    Subclasses override _encode(x_seq) -> (batch, 64).
    forward() returns (pred, None, None) to match DualCircuit_PI_HLNN's 3-tuple.
    """
    def __init__(self, dropout=0.2):
        super().__init__()
        self.fp_cond = nn.Sequential(nn.Linear(67, 64), nn.ReLU())
        self.classifier = nn.Sequential(
            nn.Linear(64, 32), nn.SiLU(), nn.Dropout(dropout),
            nn.Linear(32, 1), nn.Sigmoid(),
        )

    def _encode(self, x_seq):
        raise NotImplementedError

    def _condition_fp(self, cir_emb, fp_features):
        if fp_features is not None:
            return self.fp_cond(torch.cat([cir_emb, fp_features], dim=1))
        return cir_emb

    def embed(self, x_seq, fp_features=None):
        return self._condition_fp(self._encode(x_seq), fp_features)

    def forward(self, x_seq, fp_features=None, return_dynamics=False):
        emb = self.embed(x_seq, fp_features=fp_features)
        pred = self.classifier(emb)
        return pred, None, None


# ==========================================
# CNN1D CLASSIFIER — Global Average Pooling readout
# ==========================================
class CNN1D_Classifier(BaseClassifier):
    """1D-CNN: Conv1d stack -> GlobalAvgPool -> 64-dim -> classifier."""
    def __init__(self, dropout=0.2):
        super().__init__(dropout=dropout)
        self.backbone = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=7, padding=3),
            nn.BatchNorm1d(16), nn.ReLU(),
            nn.Conv1d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm1d(32), nn.ReLU(),
            nn.Conv1d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64), nn.ReLU(),
            nn.AdaptiveAvgPool1d(1),
        )

    def _encode(self, x_seq):
        x = x_seq.permute(0, 2, 1)       # (B, 1, 60)
        x = self.backbone(x)              # (B, 64, 1)
        return x.squeeze(-1)              # (B, 64)


# ==========================================
# LSTM CLASSIFIER — Last hidden state readout
# ==========================================
class LSTM_Classifier(BaseClassifier):
    """LSTM: process 60-step CIR -> last hidden state h_T -> project to 64-dim."""
    def __init__(self, lstm_hidden=48, dropout=0.2):
        super().__init__(dropout=dropout)
        self.lstm = nn.LSTM(input_size=1, hidden_size=lstm_hidden,
                            num_layers=1, batch_first=True)
        self.proj = nn.Linear(lstm_hidden, 64)

    def _encode(self, x_seq):
        _, (h_n, _) = self.lstm(x_seq)    # h_n: (1, B, 48)
        h_T = h_n.squeeze(0)              # (B, 48)
        return self.proj(h_T)             # (B, 64)


# ==========================================
# TRANSFORMER ENCODER CLASSIFIER — [CLS] token readout
# ==========================================
class SinusoidalPE(nn.Module):
    """Fixed sinusoidal positional encoding."""
    def __init__(self, d_model, max_len=128):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float()
                             * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :]


class TransformerEncoder_Classifier(BaseClassifier):
    """Transformer Encoder: Linear proj -> SinusoidalPE -> [CLS] + encoder layers -> [CLS] output."""
    def __init__(self, d_model=32, nhead=4, num_layers=2, dim_feedforward=64, dropout=0.2):
        super().__init__(dropout=dropout)
        self.input_proj = nn.Linear(1, d_model)
        self.pos_enc = SinusoidalPE(d_model, max_len=128)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, d_model))
        nn.init.normal_(self.cls_token, std=0.02)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward,
            dropout=0.1, batch_first=True, activation='gelu',
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.out_proj = nn.Linear(d_model, 64)

    def _encode(self, x_seq):
        B = x_seq.size(0)
        x = self.input_proj(x_seq)                    # (B, 60, d_model)
        cls = self.cls_token.expand(B, -1, -1)         # (B, 1, d_model)
        x = torch.cat([cls, x], dim=1)                 # (B, 61, d_model)
        x = self.pos_enc(x)                             # (B, 61, d_model)
        x = self.encoder(x)                              # (B, 61, d_model)
        return self.out_proj(x[:, 0, :])                 # (B, 64) — [CLS]


print('CNN, LSTM, Transformer models defined.')

In [ ]:
# ── Model factory + verification ────────────────────────────────────────
_CLS_MAP = {
    'DualCircuit_PI_HLNN': DualCircuit_PI_HLNN,
    'CNN1D_Classifier': CNN1D_Classifier,
    'LSTM_Classifier': LSTM_Classifier,
    'TransformerEncoder_Classifier': TransformerEncoder_Classifier,
}

def build_model(model_name, config=CONFIG):
    mc = MODEL_CONFIGS[model_name]
    return _CLS_MAP[mc['class']](**mc['kwargs']).to(device)


# Verify all models
x_test = torch.randn(4, 60, 1)
f_test = torch.randn(4, 3)

print(f"{'Model':>15} | {'Params':>8} | {'pred':>12} | {'embed':>12}")
print(f"{'-'*55}")
for name in MODEL_ORDER:
    m = build_model(name)
    pred, _, _ = m(x_test.to(device), fp_features=f_test.to(device))
    emb = m.embed(x_test.to(device), fp_features=f_test.to(device))
    n_params = sum(p.numel() for p in m.parameters())
    assert pred.shape == (4, 1), f"{name}: pred shape {pred.shape}"
    assert emb.shape == (4, 64), f"{name}: embed shape {emb.shape}"
    print(f'{name:>15} | {n_params:>8,} | {str(tuple(pred.shape)):>12} | {str(tuple(emb.shape)):>12}')
    del m

print('\nAll models verified: forward returns 3-tuple, embed returns (batch, 64).')

---
## Section 3: Training & Evaluation
Model-agnostic training loop: accepts `model_name` to select architecture via `build_model()`.

In [ ]:
def train_model(X_train, y_train, X_val, y_val, F_train=None, F_val=None,
                config=CONFIG, model_name='LNN', verbose=True, seed=None):
    _seed = seed if seed is not None else config['seed']
    torch.manual_seed(_seed)
    np.random.seed(_seed)
    fp_enabled = F_train is not None

    X_tr = torch.tensor(X_train).to(device)
    y_tr = torch.tensor(y_train).unsqueeze(1).to(device)
    X_va = torch.tensor(X_val).to(device)
    y_va = torch.tensor(y_val).unsqueeze(1).to(device)

    if fp_enabled:
        F_tr = torch.tensor(F_train).to(device)
        F_va = torch.tensor(F_val).to(device)
        loader = DataLoader(TensorDataset(X_tr, y_tr, F_tr),
                            batch_size=config['batch_size'], shuffle=True)
    else:
        loader = DataLoader(TensorDataset(X_tr, y_tr),
                            batch_size=config['batch_size'], shuffle=True)

    model = build_model(model_name, config)

    criterion = nn.BCELoss()
    optimizer = optim.AdamW(model.parameters(), lr=config['lr'],
                            weight_decay=config['weight_decay'])
    T = config['max_epochs']; W = config['warmup_epochs']
    def lr_lambda(e):
        if e < W: return (e+1)/W
        return max(0.01, 0.5*(1+math.cos(math.pi*(e-W)/max(1,T-W))))
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

    history = {'train_loss':[], 'val_loss':[], 'train_acc':[], 'val_acc':[]}
    best_val_acc = 0; best_state = None; patience_counter = 0

    for epoch in range(T):
        model.train(); tl, tc, tt = 0, 0, 0
        for batch in loader:
            if fp_enabled:
                bx, by, bf = batch
            else:
                bx, by = batch; bf = None
            optimizer.zero_grad()
            pred, _, _ = model(bx, fp_features=bf)
            loss = criterion(pred, by); loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), config['grad_clip'])
            optimizer.step()
            tl += loss.item()*len(bx); tc += ((pred>0.5).float()==by).sum().item(); tt += len(bx)
        model.eval()
        with torch.no_grad():
            val_fp = F_va if fp_enabled else None
            vp, _, _ = model(X_va, fp_features=val_fp)
            vl = criterion(vp, y_va).item()
            va = ((vp>0.5).float()==y_va).float().mean().item()
        history['train_loss'].append(tl/tt); history['val_loss'].append(vl)
        history['train_acc'].append(tc/tt);  history['val_acc'].append(va)
        scheduler.step()

        if va > best_val_acc:
            best_val_acc = va; best_state = copy.deepcopy(model.state_dict()); patience_counter = 0
        else:
            patience_counter += 1

        if verbose and (epoch%10==0 or epoch==T-1):
            print(f'  Ep {epoch:>3} | Loss: {tl/tt:.4f} | Val Acc: {100*va:.2f}% | Best: {100*best_val_acc:.2f}%')
        if patience_counter >= config.get('patience', T):
            if verbose: print(f'  Early stopping at epoch {epoch}')
            break

    model.load_state_dict(best_state)
    if verbose: print(f'  Best Val Acc: {100*best_val_acc:.2f}%')
    return model, history


def evaluate(model, X, y_true, F=None):
    """Returns metrics dict for a given model and data."""
    model.eval()
    X_t = torch.tensor(X).to(device)
    F_t = torch.tensor(F).to(device) if F is not None else None
    with torch.no_grad():
        pred, _, _ = model(X_t, fp_features=F_t)
    y_prob = pred.cpu().numpy().flatten()
    y_pred = (y_prob > 0.5).astype(float)
    y_true = y_true.flatten()
    rep  = classification_report(y_true, y_pred, target_names=['LOS','NLOS'],
                                 output_dict=True, zero_division=0)
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    return {
        'acc':      rep['accuracy'],
        'f1_macro': rep['macro avg']['f1-score'],
        'f1_los':   rep['LOS']['f1-score'],
        'f1_nlos':  rep['NLOS']['f1-score'],
        'auc':      auc(fpr, tpr),
        'fpr': fpr, 'tpr': tpr,
        'cm':  confusion_matrix(y_true, y_pred),
    }

print('train_model() and evaluate() ready.')

---
## Section 4: Single 70/15/15 Split — 4 Architectures
Quick baseline on the combined 3600-sample dataset.

In [ ]:
def run_experiment(csv_name, label, model_name='LNN', config=CONFIG, seed=42):
    print(f"\n{'='*60}\nExperiment: {label} | Model: {model_name}\n{'='*60}")
    X_all, y_all, F_all = load_cir_dataset(DATA_DIR + csv_name)
    X_tr, X_tmp, y_tr, y_tmp, F_tr, F_tmp = train_test_split(
        X_all, y_all, F_all, test_size=config['val_ratio']+config['test_ratio'],
        stratify=y_all, random_state=seed)
    X_va, X_te, y_va, y_te, F_va, F_te = train_test_split(
        X_tmp, y_tmp, F_tmp,
        test_size=config['test_ratio']/(config['val_ratio']+config['test_ratio']),
        stratify=y_tmp, random_state=seed)
    print(f'  Train={len(X_tr)}, Val={len(X_va)}, Test={len(X_te)}')
    model, history = train_model(X_tr, y_tr, X_va, y_va, F_train=F_tr, F_val=F_va,
                                  config=config, model_name=model_name)
    m = evaluate(model, X_te, y_te, F=F_te)
    m['label'] = label; m['history'] = history; m['model_name'] = model_name
    n_params = sum(p.numel() for p in model.parameters())
    m['params'] = n_params
    print(f"\n  Test Acc={m['acc']:.4f} | Macro F1={m['f1_macro']:.4f} | AUC={m['auc']:.4f} | Params={n_params:,}")
    return m


# Run all 4 models on the combined dataset
results_split = {}
for model_name in MODEL_ORDER:
    results_split[model_name] = run_experiment(
        'combined_uwb_dataset.csv', 'Combined (3600)', model_name=model_name)

In [ ]:
# ── Single-split plots: 4 architectures ─────────────────────────────────
_colors = [MODEL_COLORS[mn] for mn in MODEL_ORDER]
width = 0.18
_res = [results_split[mn] for mn in MODEL_ORDER]

fig, axs = plt.subplots(1, 3, figsize=(24, 6))

# Col 0: Key metrics bar chart
ax = axs[0]
mk = ['acc', 'f1_macro', 'auc']; mn_labels = ['Accuracy', 'Macro F1', 'AUC']
for i, (r, c, mn) in enumerate(zip(_res, _colors, MODEL_ORDER)):
    vals = [r[k] for k in mk]
    bars = ax.bar(np.arange(3) + i*width, vals, width, label=mn, color=c, alpha=0.85)
    for bar, v in zip(bars, vals):
        ax.text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.005,
                f'{v:.3f}', ha='center', va='bottom', fontsize=6.5, fontweight='bold')
ax.set_xticks(np.arange(3) + width*1.5); ax.set_xticklabels(mn_labels)
ax.set_ylim([0.3, 1.15]); ax.set_title('Metrics', fontweight='bold')
ax.legend(fontsize=7); ax.grid(True, alpha=0.3, axis='y')

# Col 1: ROC curves
ax = axs[1]
for r, c, mn in zip(_res, _colors, MODEL_ORDER):
    ax.plot(r['fpr'], r['tpr'], color=c, lw=2,
            label=f"{mn} (AUC={r['auc']:.3f})")
ax.plot([0,1],[0,1],'k--',lw=1,alpha=0.5)
ax.set_title('ROC Curves', fontweight='bold')
ax.set_xlabel('FPR'); ax.set_ylabel('TPR')
ax.legend(fontsize=7); ax.grid(True, alpha=0.3)

# Col 2: Val accuracy curves
ax = axs[2]
for r, c, mn in zip(_res, _colors, MODEL_ORDER):
    ax.plot(r['history']['val_acc'], color=c, lw=2, label=mn)
ax.set_title('Val Accuracy', fontweight='bold')
ax.set_xlabel('Epoch'); ax.set_ylabel('Val Accuracy')
ax.set_ylim([0.3, 1.05]); ax.legend(fontsize=7); ax.grid(True, alpha=0.3)

plt.suptitle('Stage 1: 4-Architecture Comparison — Single Split (3600 samples)',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout(); plt.show()

# Summary table
print(f"\n{'='*75}")
print(f"{'Model':<15} {'Params':>8} {'Acc':>8} {'F1':>8} {'F1-LOS':>8} {'F1-NLOS':>8} {'AUC':>8}")
print(f"{'-'*75}")
for mn in MODEL_ORDER:
    r = results_split[mn]
    print(f"{mn:<15} {r['params']:>8,} {r['acc']:>8.4f} {r['f1_macro']:>8.4f} "
          f"{r['f1_los']:>8.4f} {r['f1_nlos']:>8.4f} {r['auc']:>8.4f}")
print(f"{'='*75}")

---
## Section 5: Stratified 5-Fold Cross-Validation — 4 Architectures
More reliable estimate: averages over 5 different train/test splits per model.

In [ ]:
def run_kfold(csv_name, label, model_name='LNN', n_splits=5, config=CONFIG, seed=42):
    print(f"\n{'='*60}\n5-Fold CV: {label} | Model: {model_name}\n{'='*60}")
    X_all, y_all, F_all = load_cir_dataset(DATA_DIR + csv_name)
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed)
    fold_metrics = []
    for fold, (tv_idx, te_idx) in enumerate(skf.split(X_all, y_all)):
        X_tv, X_te = X_all[tv_idx], X_all[te_idx]
        y_tv, y_te = y_all[tv_idx], y_all[te_idx]
        F_tv, F_te = F_all[tv_idx], F_all[te_idx]
        X_tr, X_va, y_tr, y_va, F_tr, F_va = train_test_split(
            X_tv, y_tv, F_tv, test_size=0.15, stratify=y_tv, random_state=seed)
        fold_seed = seed + fold
        with contextlib.redirect_stdout(io.StringIO()):
            model, _ = train_model(X_tr, y_tr, X_va, y_va, F_train=F_tr, F_val=F_va,
                                   config=config, model_name=model_name,
                                   verbose=False, seed=fold_seed)
        fm = evaluate(model, X_te, y_te, F=F_te)
        fold_metrics.append(fm)
        collapsed = " [COLLAPSED]" if fm['acc'] <= 0.51 else ""
        print(f"  Fold {fold+1}/{n_splits} | Acc={fm['acc']:.4f} | F1={fm['f1_macro']:.4f} | AUC={fm['auc']:.4f}{collapsed}")
    summary = {'label': label, 'model_name': model_name}
    print(f"\n  {'---'*15}")
    print(f"  {'Metric':<12} {'Mean':>8} {'Std':>8}")
    print(f"  {'---'*15}")
    for key in ['acc','f1_macro','f1_los','f1_nlos','auc']:
        vals = np.array([m[key] for m in fold_metrics])
        summary[key] = {'mean': vals.mean(), 'std': vals.std(), 'all': vals.tolist()}
        print(f"  {key:<12} {vals.mean():>8.4f} {vals.std():>8.4f}")
    print(f"  {'---'*15}")
    return summary


# Run all 4 models
kfold_results = {}
for model_name in MODEL_ORDER:
    kfold_results[model_name] = run_kfold(
        'combined_uwb_dataset.csv', 'Combined (3600)', model_name=model_name)

In [ ]:
# ── K-Fold comparison plots: 4 architectures ────────────────────────────
_mk = ['acc', 'f1_macro', 'f1_los', 'f1_nlos', 'auc']
_mn = ['Accuracy', 'Macro F1', 'F1 LOS', 'F1 NLOS', 'AUC']
width = 0.18

fig, axs = plt.subplots(1, 2, figsize=(22, 7))

# Bar chart with error bars
ax = axs[0]
x = np.arange(len(_mk))
for i, mn in enumerate(MODEL_ORDER):
    kf = kfold_results[mn]
    means = [kf[m]['mean'] for m in _mk]
    stds  = [kf[m]['std']  for m in _mk]
    bars = ax.bar(x + i*width, means, width, yerr=stds,
                  label=mn, color=MODEL_COLORS[mn], alpha=0.85,
                  capsize=3, error_kw={'elinewidth': 1.2})
    for bar, m_val, s_val in zip(bars, means, stds):
        ax.text(bar.get_x()+bar.get_width()/2, bar.get_height()+s_val+0.008,
                f'{m_val:.3f}', ha='center', va='bottom', fontsize=6, fontweight='bold')
ax.set_xticks(x + width*1.5); ax.set_xticklabels(_mn, rotation=10)
ax.set_ylim([0.3, 1.2])
ax.set_title('5-Fold CV: Mean +/- Std', fontweight='bold')
ax.legend(fontsize=7); ax.grid(True, alpha=0.3, axis='y')

# Box plots
ax = axs[1]
all_data, all_colors, positions = [], [], []
gap = len(MODEL_ORDER) + 1
for mi, metric in enumerate(_mk):
    for ki, mn in enumerate(MODEL_ORDER):
        kf = kfold_results[mn]
        positions.append(mi * gap + ki)
        all_data.append(kf[metric]['all'])
        all_colors.append(MODEL_COLORS[mn])
bp = ax.boxplot(all_data, positions=positions, widths=0.65, patch_artist=True,
                medianprops={'color': 'black', 'linewidth': 2})
for patch, c in zip(bp['boxes'], all_colors):
    patch.set_facecolor(c); patch.set_alpha(0.7)
ax.set_xticks([mi * gap + 1.5 for mi in range(len(_mk))])
ax.set_xticklabels(_mn, rotation=10)
ax.set_title('Score Distribution per Fold', fontweight='bold')
ax.set_ylabel('Score'); ax.grid(True, alpha=0.3, axis='y')
legend_patches = [Patch(facecolor=MODEL_COLORS[mn], alpha=0.7, label=mn) for mn in MODEL_ORDER]
ax.legend(handles=legend_patches, fontsize=7)

plt.suptitle('Stage 1: 4-Architecture Comparison — Stratified 5-Fold CV (3600 samples)',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout(); plt.show()

# Summary table
print(f"\n{'='*80}")
print(f"{'Model':<15} {'Acc':>14} {'Macro F1':>14} {'F1-LOS':>14} {'F1-NLOS':>14} {'AUC':>14}")
print(f"{'-'*80}")
for mn in MODEL_ORDER:
    kf = kfold_results[mn]
    print(f"{mn:<15}", end="")
    for m in _mk:
        print(f" {kf[m]['mean']:>6.4f}+/-{kf[m]['std']:.4f}", end="")
    print()
print(f"{'='*80}")